In [5]:
import cv2
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from einops.layers.torch import Rearrange
from einops import rearrange
from facenet_pytorch import MTCNN
from self_attention_cv import TransformerEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import face_alignment
import requests
import warnings
warnings.filterwarnings("ignore")

In [6]:
!ls 'Videos7439' | wc -l

7439


In [7]:
!ls 'Spectrograms7439' | wc -l

7439


In [8]:
videos_folder = 'Videos7439'
spectrograms_folder = 'Spectrograms7439'

In [9]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.features = models.resnet18(weights='DEFAULT')
        # Modify the first layer to accept 1 channel input (for grayscale spectrograms)
        self.features.conv1 = nn.Conv2d(2, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Modify the final layer to output desired feature size
        self.features.fc = nn.Linear(self.features.fc.in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.features(x)
        x = self.softmax(x)
        return x

In [10]:
def extract_frame(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    mid_frame_index = frame_count // 2  # Index of the frame in the middle of the video
    cap.set(cv2.CAP_PROP_POS_FRAMES, mid_frame_index)
    ret, frame = cap.read()
    if ret:
        cap.release()
        return frame
    else:
        cap.release()
        return None

In [11]:
def detect_face(frame):
    mtcnn = MTCNN()
    boxes, _ = mtcnn.detect(frame)
    if boxes is not None:
        # Assuming only one face in the frame
        box = boxes[0]
        x1, y1, x2, y2 = box
        # Crop the frame to the detected face
        cropped_frame = frame[int(y1):int(y2), int(x1):int(x2)]
        return cropped_frame
    else:
        return None

In [12]:
def preprocess_image(frame):
    # Convert the frame to a PIL Image
    frame_pil = Image.fromarray(frame.astype('uint8'))

    # Convert the image to grayscale
    frame_pil = frame_pil.convert('L')

    # Resize and normalize the frame
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229]),  # For grayscale images, only 1 channel
    ])
    img_tensor = transform(frame_pil)
    return img_tensor

In [13]:
def preprocess_spectrogram(image_path):
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match ResNet input size
        transforms.ToTensor(),           # Convert to tensor
    ])
    img_tensor = transform(img)
    # img_tensor = img_tensor.unsqueeze(0)  # Add batch dimension
    return img_tensor

In [14]:
def load_spectrogram_dataset(spectrograms_folder, skipped_files):
    X = []
    y = []
    # List all files in the input folder
    files = sorted(os.listdir(spectrograms_folder))
    # Iterate over files in the folder
    for filename in tqdm(files):
        if filename.endswith(".png") and filename[:-3] not in skipped_files:  # Assuming mel spectrograms are stored as PNG files
            input_path = os.path.join(spectrograms_folder, filename)
            img_tensor = preprocess_spectrogram(input_path)
            X.append(img_tensor)
            # Extract label from filename (assuming filename is in format "abc_IEO_label_xyz.png")
            label = filename.split("_")[2]
            if label == "HAP":
                y.append(0)
            elif label == "SAD":
                y.append(1)
            elif label == "ANG":
                y.append(2)
            elif label == "DIS":
                y.append(3)
            elif label == "FEA":
                y.append(4)
            elif label == "NEU":
                y.append(5)
    return X, y

In [15]:
def load_dataset(videos_folder):
    X = []
    y = []
    skipped_files = []
    video_files = [file for file in sorted(os.listdir(videos_folder)) if file.endswith(".flv")]
    for video_file in tqdm(video_files):
        video_path = os.path.join(videos_folder, video_file)
        frame = extract_frame(video_path)
        if frame is not None:
            cropped_face = detect_face(frame)
            if cropped_face is not None:
                preprocessed_face = preprocess_image(cropped_face)
                X.append(preprocessed_face)
                label = video_file.split("_")[2].split(".")[0]  # Adjusted to handle different file extensions
                if label == "HAP":
                    y.append(0)
                elif label == "SAD":
                    y.append(1)
                elif label == "ANG":
                    y.append(2)
                elif label == "DIS":
                    y.append(3)
                elif label == "FEA":
                    y.append(4)
                elif label == "NEU":
                    y.append(5)
            else:
                print(f"No face detected in {video_file}. Skipping.")
                skipped_files.append(video_file[:-3])
        else:
            print(f"Failed to extract frame from {video_file}. Skipping.")
            skipped_files.append(video_file[:-3])
    return X, y, skipped_files

In [16]:
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, X1, X2, y):
        self.X1 = X1
        self.X2 = X2
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        img1 = self.X1[idx]
        img2 = self.X2[idx]
        label = self.y[idx]
        # Convert numpy arrays to tensors
        img1 = torch.tensor(img1, dtype=torch.float32)
        img2 = torch.tensor(img2, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)
        # Concatenate images along the channel dimension
        concatenated_img = torch.cat((img1, img2), dim=0)  # Note: Changed dim=2 to dim=0 for channel concatenation
        return concatenated_img, label


In [17]:
def train_model(model, criterion, optimizer, train_loader, device):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

In [18]:
def test_model(model, criterion, test_loader, device):
    model.eval()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)
    epoch_loss = running_loss / len(test_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

In [19]:

X = np.load('X_resnet.npy', mmap_mode='r')
y = np.load('y_resnet.npy', mmap_mode='r')
X_spec = np.load('X_spec_resnet.npy', mmap_mode='r')
y_spec = np.load('y_spec_resnet.npy', mmap_mode='r')


print(f"Total number of samples: {len(X)}")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(f"Number of train samples (video): {len(X_train)}", f"Number of test samples: {len(X_test)}")
X_train_spec, X_test_spec, y_train_spec, y_test_spec = train_test_split(X_spec, y_spec, test_size=0.3, random_state=42)
print(f"Number of train samples (audio): {len(X_train_spec)}", f"Number of test samples: {len(X_test_spec)}")



Total number of samples: 7429
Number of train samples (video): 5200 Number of test samples: 2229
Number of train samples (audio): 5200 Number of test samples: 2229


In [20]:
# Initialize the model
model = CNN(num_classes=6)  # 3 classes for HAPPY, SAD, ANGRY
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
_lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=_lr)

# Concatenate datasets
train_dataset = ConcatDataset(X_train, X_train_spec, y_train)
test_dataset = ConcatDataset(X_test, X_test_spec, y_test)

# Create data loaders
_bs = 32
# train_loader = torch.utils.data.DataLoader(list(zip(X_train, y_train)), batch_size=_bs, shuffle=True)
# test_loader = torch.utils.data.DataLoader(list(zip(X_test, y_test)), batch_size=_bs)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=_bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=_bs)


print(f"Batch size: {_bs}", f"lr: {_lr}")

Batch size: 32 lr: 0.001


In [21]:
# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    print("Epoch " + str(epoch))
    train_loss, train_accuracy = train_model(model, criterion, optimizer, train_loader, device)
    test_loss, test_accuracy = test_model(model, criterion, test_loader, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 0


100%|██████████| 70/70 [00:01<00:00, 47.48it/s]


Epoch 1/50, Train Loss: 1.6507, Train Accuracy: 0.3719, Test Loss: 1.6276, Test Accuracy: 0.3975
Epoch 1


100%|██████████| 70/70 [00:01<00:00, 48.07it/s]


Epoch 2/50, Train Loss: 1.5047, Train Accuracy: 0.5292, Test Loss: 1.4963, Test Accuracy: 0.5410
Epoch 2


...

Epoch 48/50, Train Loss: 1.1112, Train Accuracy: 0.9317, Test Loss: 1.3054, Test Accuracy: 0.7362
Epoch 48


100%|██████████| 70/70 [00:01<00:00, 47.87it/s]


Epoch 49/50, Train Loss: 1.1020, Train Accuracy: 0.9412, Test Loss: 1.2934, Test Accuracy: 0.7479
Epoch 49


100%|██████████| 70/70 [00:01<00:00, 48.07it/s]

Epoch 50/50, Train Loss: 1.1050, Train Accuracy: 0.9379, Test Loss: 1.3074, Test Accuracy: 0.7326


In [22]:
torch.save(model.state_dict(), 'ResNet18_audio_video_'+str(num_epochs)+'_'+str(_bs)+'_'+str(_lr))